In [1]:
from utils.binaries import *
from utils.plotting import *
from utils.auger.fd import XYScanner
from datetime import datetime
from utils import CONSTANTS


14:57:04 (   +4.2s) [INFO   ] -- import logging
14:57:04 (  +181ms) [INFO   ] -- import numpy as np
14:57:04 (   +12ms) [INFO   ] -- import uncertainties
14:57:04 (  +430ms) [INFO   ] -- import pandas as pd
14:57:04 (    +2ms) [WARNING] -- pandas support to be dropped at some point!
14:57:04 (    +2ms) [INFO   ] -- import binaries.tools as tools
14:57:05 (  +470ms) [INFO   ] -- import matplotlib.pyplot as plt
14:57:06 (  +585ms) [INFO   ] -- import seaborn as so
14:57:06 (    +4ms) [INFO   ] -- set MONI_PATH = PosixPath('/cr/work/filip/monit_and_sd')
14:57:06 (    +1ms) [INFO   ] -- set HIST_PATH = PosixPath('/cr/work/filip/monit_and_sd')
14:57:06 (    +1ms) [INFO   ] -- set PLOT_PATH = PosixPath('/cr/data01/filip/plots')
14:57:06 (    +1ms) [INFO   ] -- set DATA_PATH = PosixPath('/cr/data01/filip/Data')
14:57:06 (    +1ms) [INFO   ] -- set SCAN_PATH = PosixPath('/cr/data01/filip/xy-calibration')
14:57:06 (    +1ms) [INFO   ] -- set OFLN_PATH = PosixPath('/cr/data01/filip/offline/insta

In [2]:
# TODO pixel plot

In [3]:
from matplotlib.backends.backend_pdf import PdfPages

campaign_dates = ["2022-10", "2023-10", "2023-11", "2024-11"]
create_empty = lambda s: np.array([[[np.nan] * 440] * 4] * s)

for site in ["LL", "LM", "LA", "CO", "HE"]:

    pdf1 = PdfPages(CONSTANTS.PLOT_PATH / f"xy_ridgeplots/{site}_distribution.pdf")
    pdf2 = PdfPages(CONSTANTS.PLOT_PATH / f"xy_ridgeplots/{site}_pixel_plot_normalized.pdf")
    pdf3 = PdfPages(CONSTANTS.PLOT_PATH / f"xy_ridgeplots/{site}_pixel_plot_unnormalized.pdf")

    tel_number = 3 if site == "HE" else 6
    data_raw = create_empty(tel_number)
    data_gain = create_empty(tel_number)
    data_lcu = create_empty(tel_number)
    data_ratio = create_empty(tel_number)
    data_old = create_empty(tel_number)
    data_gaetano = create_empty(tel_number)

    for i in range(tel_number):

        extension = site.lower() if site.lower() != "he" else f"he{i+1}"
        lcu_time = tools.pickle_load(f"/cr/tempdata01/filip/LCUDATA/data/x.{extension}")
        lcu_data = tools.pickle_load(f"/cr/tempdata01/filip/LCUDATA/data/y.{extension}")

        tel = XYScanner.Telescope(f"{site}{i+1}")

        for run in tel:
            if run.year_month_day.startswith("2019"): continue
            if run.source != "OLO": continue
            if run.stepsize != 6: continue
            if run.year_month_day.startswith("2024-12"): 
                run.year_month_day = "2024-11-30"
            j = campaign_dates.index(run.year_month_day[:-3])

            this_raw = run.get_data("XY")
            this_preXY = run.get_data("preXY")
            this_postXY = run.get_data("postXY")
            if this_raw is None \
            or this_preXY is None \
            or this_postXY is None: continue

            this_gain = this_raw / (0.5 * (this_preXY + this_postXY))
            this_ratio = run.get_data("ratio")
            this_old = this_ratio / this_gain

            # raise a warning if found closest index is last index
            # likely we need to get more data from lyon LCU files
            # don't forget to recreate {x,y}.* files via read_dat.ipynb
            run_time = datetime.strptime(run.year_month_day, "%Y-%m-%d")
            idx = tools.closest(lcu_time, run_time)

            if idx + 1 == len(lcu_time): raise ValueError(f"get more LCU data for {site = }!")
            this_lcu = this_raw / lcu_data[idx]
            
            data_raw[i, j] = this_raw
            data_gain[i, j] = this_gain
            data_lcu[i, j] = this_lcu
            data_ratio[i, j] = this_ratio
            data_old[i, j] = this_old
            data_gaetano[i, j] = this_ratio / lcu_data[idx]

    
    # raw data
    if True:
        raw = plot.ridgeplot2d(data_raw, 
                            cbar_ticks=campaign_dates, 
                            labels=[f"{site}{i}" for i in range(1, tel_number+1)],
                            xlabel=r"XY calib. / $\frac{\gamma}{\mathrm{ADC}}$",
                            ylabel="Campaign / YYYY-MM",)
        pdf1.savefig(raw)
        plt.close(raw)

        raw_pixels_normalized = aperture_grid(data_raw,
                                              xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                              ylabel=campaign_dates, vmin='min', vmax='max',
                                              cbar_label=r"XY calib. / $\frac{\gamma}{\mathrm{ADC}}$",)
        pdf2.savefig(raw_pixels_normalized)
        plt.close(raw_pixels_normalized)

        raw_pixels_unnormalized = aperture_grid(data_raw,
                                               xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                               ylabel=campaign_dates,
                                               cbar_label=r"XY calib. / $\frac{\gamma}{\mathrm{ADC}}$",)
        pdf3.savefig(raw_pixels_unnormalized)
        plt.close(raw_pixels_unnormalized)
        

    # gain
    if True:
        gain = plot.ridgeplot2d(data_gain * 1e6, 
                                cbar_ticks=campaign_dates, 
                                labels=[f"{site}{i}" for i in range(1, tel_number+1)],
                                xlabel=r"$10^6\times$XY calib. / FDEventSum",
                                ylabel="Campaign / YYYY-MM",)
        pdf1.savefig(gain)
        plt.close(gain)

        gain_pixels_normalized = aperture_grid(data_gain * 1e6, 
                                               xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                               ylabel=campaign_dates, vmin="min", vmax='max',
                                               cbar_label=r"$10^6\times$XY calib. / FDEventSum",)
        pdf2.savefig(gain_pixels_normalized)
        plt.close(gain_pixels_normalized)

        gain_pixels_unnormalized = aperture_grid(data_gain * 1e6, 
                                                 xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                 ylabel=campaign_dates,
                                                 cbar_label=r"$10^6\times$XY calib. / FDEventSum",)
        pdf3.savefig(gain_pixels_unnormalized)
        plt.close(gain_pixels_unnormalized)


    # lcu
    if True:
        lcu = plot.ridgeplot2d(data_lcu * 1e6, 
                            cbar_ticks=campaign_dates, 
                            labels=[f"{site}{i}" for i in range(1, tel_number+1)],
                            xlabel=r"$10^6\times$XY calib. / LCU",
                            ylabel="Campaign / YYYY-MM",)
        pdf1.savefig(lcu)
        plt.close(lcu)

        lcu_pixels_normalized = aperture_grid(data_lcu * 1e6,
                                              xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                              ylabel=campaign_dates, vmin='min', vmax='max',
                                              cbar_label=r"$10^6\times$XY calib. / LCU",)
        pdf2.savefig(lcu_pixels_normalized)
        plt.close(lcu_pixels_normalized)

        lcu_pixels_unnormalized = aperture_grid(data_lcu * 1e6,
                                                xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                ylabel=campaign_dates,
                                                cbar_label=r"$10^6\times$XY calib. / LCU",)
        pdf3.savefig(lcu_pixels_unnormalized)
        plt.close(lcu_pixels_unnormalized)


    # ratio
    if True:
        ratio = plot.ridgeplot2d(data_ratio, 
                                cbar_ticks=campaign_dates, 
                                labels=[f"{site}{i}" for i in range(1, tel_number+1)],
                                xlabel=r"XY calib. / std. calib.",
                                ylabel="Campaign / YYYY-MM",)
        pdf1.savefig(ratio)
        plt.close(ratio)

        ratio_pixels_normalized = aperture_grid(data_ratio,
                                                xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                ylabel=campaign_dates, vmin="min", vmax="max", 
                                                cbar_label=r"XY calib. / std. calib.",)
        pdf2.savefig(ratio_pixels_normalized)
        plt.close(ratio_pixels_normalized)

        ratio_pixels_unnormalized = aperture_grid(data_ratio,
                                                  xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                  ylabel=campaign_dates, 
                                                  cbar_label=r"XY calib. / std. calib.",)
        pdf2.savefig(ratio_pixels_unnormalized)
        plt.close(ratio_pixels_unnormalized)


    # old
    if True:
        old = plot.ridgeplot2d(data_old * 1e6, 
                            cbar_ticks=campaign_dates, 
                            labels=[f"{site}{i}" for i in range(1, tel_number+1)],
                            xlabel=r"$10^6$ XY calib. / (std. calib. $\times$ FDEventSum)",
                            ylabel="Campaign / YYYY-MM",)
        pdf1.savefig(old)
        plt.close(old)

        old_pixels_normalized = aperture_grid(data_old * 1e6,
                                              xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                              ylabel=campaign_dates, vmin="min", vmax="max", 
                                              cbar_label=r"$10^6$ XY calib. / (std. calib. $\times$ FDEventSum)",)
        pdf2.savefig(old_pixels_normalized)
        plt.close(old_pixels_normalized)

        old_pixels_unnormalized = aperture_grid(data_old * 1e6,
                                                xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                ylabel=campaign_dates,
                                                cbar_label=r"$10^6$ XY calib. / (std. calib. $\times$ FDEventSum)",)
        pdf2.savefig(old_pixels_unnormalized)
        plt.close(old_pixels_unnormalized)


    # gaetano
    if True:
        gaetano = plot.ridgeplot2d(data_gaetano * 1e6, 
                                cbar_ticks=campaign_dates, 
                                labels=[f"{site}{i}" for i in range(1, tel_number+1)],
                                xlabel=r"$10^6 \times$ XY calib. / (std. calib. $\times$ LCU)",
                                ylabel="Campaign / YYYY-MM",)
        pdf1.savefig(gaetano)
        plt.close(gaetano)

        gaetano_pixels_normalized = aperture_grid(data_gaetano * 1e6, 
                                                  xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                  ylabel=campaign_dates, vmin="min", vmax="max", 
                                                  cbar_label=r"$10^6 \times$ XY calib. / (std. calib. $\times$ LCU)")
        pdf2.savefig(gaetano_pixels_normalized)
        plt.close(gaetano_pixels_normalized)

        gaetano_pixels_unnormalized = aperture_grid(data_gaetano * 1e6, 
                                                    xlabel=[f"{site}{i}" for i in range(1, tel_number+1)],
                                                    ylabel=campaign_dates,
                                                    cbar_label=r"$10^6 \times$ XY calib. / (std. calib. $\times$ LCU)")
        pdf3.savefig(gaetano_pixels_normalized)
        plt.close(gaetano_pixels_normalized)

    pdf1.close()
    pdf2.close()
    pdf3.close()

NameError: name 'aperture_grid' is not defined

In [4]:
from matplotlib.colors import Normalize, Colormap
from matplotlib.gridspec import GridSpec
from matplotlib.colors import BoundaryNorm
from matplotlib.cm import ScalarMappable
from matplotlib import dates
from utils.auger.fd.XYScanner import PixelPlot

def aperture_grid(data: np.ndarray, **kwargs) -> plt.Figure:

    cols, rows, npix = data.shape

    three_sigma_cut = lambda d: d[np.abs(d - np.nanmean(d))/np.nanstd(d) < 3]
    vmin, vmax = kwargs.get("vmin", None), kwargs.get("vmax", None)
    match vmin:
        case "min": vmin = np.min(three_sigma_cut(data))
        case None: vmin = None
        case _: pass
    match vmax:
        case "max": vmax = np.max(three_sigma_cut(data))
        case None: vmax = None
        case _: pass
        
    cmap = plt.get_cmap(kwargs.get("cmap", "viridis"))
    cbar_label = kwargs.get("cbar_label", "data")
    ylabel = kwargs.get("ylabel", [""] * rows)
    xlabel = kwargs.get("xlabel", [""] * cols)


    fig = plt.figure()
    gs = GridSpec(rows, cols + 1, fig,
                  hspace=0.01, wspace=0.01,
                  width_ratios=[1 for _ in range(cols)] + [0.02 * cols])

    for i in range(rows):
        for j in range(cols):

            ax = fig.add_subplot(gs[i, j])
            if sum(np.isnan(data[j, i])) == 440:
                ax.set_aspect("equal")
                ax.axis("off")
            else:
                PixelPlot(data[j, i], ax, cmap=cmap, vmin=vmin, vmax=vmax, lw=0.4)
            
            if not j: 
                ax.text(-0.1, 0.5,
                        ylabel[i],
                        horizontalalignment='center',
                        verticalalignment='center',
                        transform=ax.transAxes,
                        rotation=90)
            if not i: 
                ax.text(0.5, 1.1,
                    xlabel[j],
                    horizontalalignment='center',
                    verticalalignment='center',
                    transform=ax.transAxes)

    if vmin is not None or vmax is not None:
        cax = fig.add_subplot(gs[:, -1])
        mappable = ScalarMappable(Normalize(vmin, vmax, cmap.N), cmap)
        plt.colorbar(mappable, cax=cax, label=cbar_label)

    return fig

In [ ]:
data_gaetano